In [1]:
!pip install tweet-preprocessor

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re #regular expressions
import string 

from nltk.corpus import stopwords
from nltk import word_tokenize
import preprocessor as p
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

reuse = False

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/coronavirus-covid19-tweets-early-april/2020-04-11 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-early-april/2020-03-29 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-early-april/2020-04-06 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-early-april/2020-04-09 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-early-april/2020-04-05 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-early-april/2020-04-12 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-early-april/2020-03-30 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-early-april/2020-04-14 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-early-april/2020-04-08 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-early-april/2020-04-03 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-early-april/2020-04-04 Coronavirus Tweets.CSV
/kaggle/input/coronavirus-covid19-tweets-ea

In [3]:
def join_df(path):
    data = pd.DataFrame()
    for sub_path in os.listdir(path):
        print(f"In path : {path}{sub_path}")
        for file in os.listdir(path+sub_path):
            temp = pd.read_csv(path+sub_path+'/'+file)
            temp = temp[(temp.country_code=='IN') & (temp.lang=='en')]
            data = pd.concat([data,temp])
    return data
        

In [4]:
path = '/kaggle/input/'

if os.path.exists('/kaggle/working/main.csv'):    
    print('Loading existing dataset')
    df = pd.read_csv('/kaggle/working/main.csv')
    reuse = True
else:
    print('Creating main dataset')
    df = join_df(path)
    df.to_csv("/kaggle/working/main.csv")
    
    
print(df.head())
print(df.shape)


Creating main dataset
In path : /kaggle/input/coronavirus-covid19-tweets-early-april
In path : /kaggle/input/coronavirus-covid19-tweets-late-april
                status_id              user_id            created_at  \
504   1248762990792343552           2332176984  2020-04-11T00:01:12Z   
4215  1248767251202764801  1238157488890843136  2020-04-11T00:18:08Z   
4505  1248767559219818497   757074841606492161  2020-04-11T00:19:21Z   
4785  1248767857707462659           1362617293  2020-04-11T00:20:32Z   
5261  1248768398797860864             76961220  2020-04-11T00:22:41Z   

      screen_name                                               text  \
504   Dattamupade  FICCI-UPSC organized a webinar on Challenges &...   
4215     xcom3333  @PMOIndia\n@narendramodi\n@ysjagan\n@GVMC_OFFI...   
4505    swachhhyd  CO2 emissions could fall by 5% in 2020. \n#cor...   
4785   mahfooz_16  Please co-operate the district administration ...   
5261  souravipaul  #COVID19 pandemic is causing more than 1.

In [5]:
# checking for missing values
df.isnull().sum()

status_id                   0
user_id                     0
created_at                  0
screen_name                 0
text                        0
source                      0
reply_to_status_id      36725
reply_to_user_id        34670
reply_to_screen_name    34670
is_quote                    0
is_retweet                  0
favourites_count            0
retweet_count               0
country_code                0
place_full_name             0
place_type                  0
followers_count             0
friends_count               0
account_lang            41618
account_created_at          0
verified                    0
lang                        0
dtype: int64

In [6]:
#drop colunmns with null values and non relevant columns 

df.drop(df.columns[df.isnull().any()],axis=1,inplace=True)
if reuse:
    
    df.drop(['Unnamed: 0','favourites_count','retweet_count','country_code','followers_count','friends_count','verified','account_created_at','lang'],axis=1,inplace=True)
else:
    df.drop(['favourites_count','retweet_count','country_code','followers_count','friends_count','verified','account_created_at','lang'],axis=1,inplace=True)

    
df.head()

,status_id,user_id,created_at,screen_name,text,source,is_quote,is_retweet,place_full_name,place_type
504,1248762990792343552,2332176984,2020-04-11T00:01:12Z,Dattamupade,FICCI-UPSC organized a webinar on Challenges &...,Twitter for Android,False,False,"Pune, India",city
4215,1248767251202764801,1238157488890843136,2020-04-11T00:18:08Z,xcom3333,@PMOIndia\n@narendramodi\n@ysjagan\n@GVMC_OFFI...,Twitter for Android,False,False,"Vishakhapatnam, India",city
4505,1248767559219818497,757074841606492161,2020-04-11T00:19:21Z,swachhhyd,CO2 emissions could fall by 5% in 2020. \n#cor...,Twitter for Android,False,False,"Hyderabad, India",city
4785,1248767857707462659,1362617293,2020-04-11T00:20:32Z,mahfooz_16,Please co-operate the district administration ...,Twitter for Android,True,False,"Tanda, India",city
5261,1248768398797860864,76961220,2020-04-11T00:22:41Z,souravipaul,#COVID19 pandemic is causing more than 1.6 bil...,Twitter for iPhone,False,False,"New Delhi, India",city


In [7]:
# processing timestamp

datetime = df['created_at'].str.split(pat='T')
datetime = list(map(lambda x: " ".join(x),datetime))
df['datetime'] = pd.to_datetime(datetime)
df.drop('created_at',axis=1,inplace=True)
df['datetime']

504      2020-04-11 00:01:12+00:00
4215     2020-04-11 00:18:08+00:00
4505     2020-04-11 00:19:21+00:00
4785     2020-04-11 00:20:32+00:00
5261     2020-04-11 00:22:41+00:00
                    ...           
343782   2020-04-28 23:01:45+00:00
345665   2020-04-28 23:09:43+00:00
347128   2020-04-28 23:15:53+00:00
347537   2020-04-28 23:17:39+00:00
348296   2020-04-28 23:21:08+00:00
Name: datetime, Length: 41618, dtype: datetime64[ns, UTC]

In [8]:
#cleaning the tweets function

#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)


#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

def clean_tweets(tweet):
 
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
    #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
#filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
#looping through conditions
    for w in word_tokens:
#check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)





In [9]:
#replace tabs and spaces

df['text']= df['text'].replace({'\n':" ","\t":" "})

df['text']

504       FICCI-UPSC organized a webinar on Challenges &...
4215      @PMOIndia\n@narendramodi\n@ysjagan\n@GVMC_OFFI...
4505      CO2 emissions could fall by 5% in 2020. \n#cor...
4785      Please co-operate the district administration ...
5261      #COVID19 pandemic is causing more than 1.6 bil...
                                ...                        
343782    @BeingSalmanKhan @BabaSiddique @zeeshan_iyc We...
345665    AC on #Corona not really gone! We've all heard...
347128    Kejriwal govt launches a single window for all...
347537    https://t.co/d6iRBjHAc9  Rapper @HIMANSHU talk...
348296    These same journalists would blame the govt. o...
Name: text, Length: 41618, dtype: object

In [10]:
#get the hastags

#using regular expressions
hashtags = df['text'].apply(lambda x: re.findall(r'\B#\w*[a-zA-Z]+\w*',x))
print(hashtags)
print(len(hashtags))
df['hashtag'] = hashtags

504                           [#COVID19]
4215               [#lockdown, #covid19]
4505            [#coronavirus, #climate]
4785                 [#StayHomeStaySafe]
5261                          [#COVID19]
                       ...              
343782                    [#coronavirus]
345665           [#Corona, #Coronavirus]
347128    [#COVID19, #DelhiFightsCorona]
347537                       [#Covid_19]
348296                        [#COVID19]
Name: text, Length: 41618, dtype: object
41618


In [11]:
df.reset_index(drop=True,inplace=True)

In [12]:
#clean the tweets
#remove mentions, emoticons, punctuations

#dask dataframes for parallel computing
import dask.dataframe as dsk
import multiprocessing as mp
from dask.multiprocessing import get


ddf = dsk.from_pandas(df,npartitions=4*mp.cpu_count())

#method 1
#ddf['text'] = ddf['text'].apply(p.clean,meta=('clean_text','str')).compute(scheduler='processes') #preprocessing using prebuilt functions


#method 2
clean_text= ddf['text'].map_partitions(lambda x: x.apply(clean_tweets),meta=('clean_text','str')).compute(scheduler='processes')

ddf['text'] = clean_text
# clean_text = ddf['text'].apply(clean_tweets,meta=('clean_text','str')).compute(scheduler='processes') #preprocessing using user defined functions

# ddf['text']= clean_text

In [13]:
clean_text = ddf['text'].apply(p.clean,meta=('clean_text','str')).compute(scheduler='processes') #preprocessing using user defined functions

ddf['text'] = clean_text

ddf.head()

,status_id,user_id,screen_name,text,source,is_quote,is_retweet,place_full_name,place_type,datetime,hashtag
0,1248762990792343552,2332176984,Dattamupade,FICCI-UPSC organized webinar Challenges amp Op...,Twitter for Android,False,False,"Pune, India",city,2020-04-11 00:01:12+00:00,[#COVID19]
1,1248767251202764801,1238157488890843136,xcom3333,PMOIndia narendramodi ysjagan GVMC_OFFICIAL AP...,Twitter for Android,False,False,"Vishakhapatnam, India",city,2020-04-11 00:18:08+00:00,"[#lockdown, #covid19]"
2,1248767559219818497,757074841606492161,swachhhyd,CO2 emissions could fall coronavirus climate h...,Twitter for Android,False,False,"Hyderabad, India",city,2020-04-11 00:19:21+00:00,"[#coronavirus, #climate]"
3,1248767857707462659,1362617293,mahfooz_16,Please co-operate district administration keep...,Twitter for Android,True,False,"Tanda, India",city,2020-04-11 00:20:32+00:00,[#StayHomeStaySafe]
4,1248768398797860864,76961220,souravipaul,COVID19 pandemic causing billion children scho...,Twitter for iPhone,False,False,"New Delhi, India",city,2020-04-11 00:22:41+00:00,[#COVID19]


In [14]:
#getting sentiment scores and polarity
from textblob import TextBlob

def get_sentiment(text):
    blob = TextBlob(text)
    return [blob.sentiment.polarity, blob.sentiment.subjectivity]




In [15]:
#converting back to pandas 
df = ddf.compute(scheduler='processes')
df.head()

,status_id,user_id,screen_name,text,source,is_quote,is_retweet,place_full_name,place_type,datetime,hashtag
0,1248762990792343552,2332176984,Dattamupade,FICCI-UPSC organized webinar Challenges amp Op...,Twitter for Android,False,False,"Pune, India",city,2020-04-11 00:01:12+00:00,[#COVID19]
1,1248767251202764801,1238157488890843136,xcom3333,PMOIndia narendramodi ysjagan GVMC_OFFICIAL AP...,Twitter for Android,False,False,"Vishakhapatnam, India",city,2020-04-11 00:18:08+00:00,"[#lockdown, #covid19]"
2,1248767559219818497,757074841606492161,swachhhyd,CO2 emissions could fall coronavirus climate h...,Twitter for Android,False,False,"Hyderabad, India",city,2020-04-11 00:19:21+00:00,"[#coronavirus, #climate]"
3,1248767857707462659,1362617293,mahfooz_16,Please co-operate district administration keep...,Twitter for Android,True,False,"Tanda, India",city,2020-04-11 00:20:32+00:00,[#StayHomeStaySafe]
4,1248768398797860864,76961220,souravipaul,COVID19 pandemic causing billion children scho...,Twitter for iPhone,False,False,"New Delhi, India",city,2020-04-11 00:22:41+00:00,[#COVID19]


In [16]:
polarity,subjectivity = [], []
for i in range(len(df)):
    pol , sub = get_sentiment(df['text'][i])
    polarity.append(pol)
    subjectivity.append(sub)
    
df['polarity'] = polarity
df['subjectivity'] = subjectivity
df.head()

,status_id,user_id,screen_name,text,source,is_quote,is_retweet,place_full_name,place_type,datetime,hashtag,polarity,subjectivity
0,1248762990792343552,2332176984,Dattamupade,FICCI-UPSC organized webinar Challenges amp Op...,Twitter for Android,False,False,"Pune, India",city,2020-04-11 00:01:12+00:00,[#COVID19],0.000,0.000
1,1248767251202764801,1238157488890843136,xcom3333,PMOIndia narendramodi ysjagan GVMC_OFFICIAL AP...,Twitter for Android,False,False,"Vishakhapatnam, India",city,2020-04-11 00:18:08+00:00,"[#lockdown, #covid19]",0.125,0.375
2,1248767559219818497,757074841606492161,swachhhyd,CO2 emissions could fall coronavirus climate h...,Twitter for Android,False,False,"Hyderabad, India",city,2020-04-11 00:19:21+00:00,"[#coronavirus, #climate]",0.000,0.000
3,1248767857707462659,1362617293,mahfooz_16,Please co-operate district administration keep...,Twitter for Android,True,False,"Tanda, India",city,2020-04-11 00:20:32+00:00,[#StayHomeStaySafe],0.000,0.000
4,1248768398797860864,76961220,souravipaul,COVID19 pandemic causing billion children scho...,Twitter for iPhone,False,False,"New Delhi, India",city,2020-04-11 00:22:41+00:00,[#COVID19],-0.700,0.800


In [17]:
#get the unique hastags

unique_hashtag = set([x for i in range(len(df)) for x in df['hashtag'][i]])
print(len(unique_hashtag))

28827


In [18]:
df.to_csv('/kaggle/working/processed.csv')